In [ ]:
import pandas as pd
labels_dataset = pd.read_csv('eval-dataset-with-labels.csv')
labels_dataset.head()

,Unnamed: 0,id,source,recover,source_score,recover_score,class
0,0,306d294a-0134-4fe3-b985-dba06380fc26,idk about you guys but i got every achievement...,idk about you guys but i think i got got every...,0.001786,0.968019,non-suicide
1,1,ee9577bc-781f-4082-bf48-5079d329e692,dude my teacher just gave us smarties this is ...,dude my teacher just perhaps gave us smarties ...,0.001046,0.958875,non-suicide
2,2,a6d95001-4daf-4744-b326-26f044c8ff06,coincidentally i see one of my closer friends ...,coincidentally i see one of my closer friends ...,0.001560,0.747658,non-suicide
3,3,2ec5dcec-c564-45f3-b2d0-0d45906b4eb5,"didn ' t ask me for permission, just shared it...","didn ' t ask me for permission, just shared it...",0.001556,0.926769,non-suicide
4,4,3314d13a-f7e5-4284-aff9-e1ee5b3ef10b,the bee community is supporting you in this ca...,the bee community is somewhat supporting you t...,0.000900,0.969288,non-suicide


In [7]:
import json

# Read JSON Lines file (each line is a separate JSON object)
model_output = []
with open('seed243_step0.json', 'r') as f:
    for line in f:
        model_output.append(json.loads(line.strip()))

print(f"Loaded {len(model_output)} records")
model_output[0] if model_output else "No data"

Loaded 2644 records


{'id': '00382f93-fd66-4d49-91fc-fc3487a9fd46',
 'recover': "[CLS] i feel depressed even though i ' m not my winter break is ending and school school starts the day after tomorrow. [SEP]",
 'reference': '[CLS] [SEP]',
 'source': "[CLS] < hedge _ up > i feel depressed even though i ' m not my winter break is ending and school starts the day after tomorrow. [SEP] [SEP]"}

In [22]:
import re

# Convert model_output to DataFrame
model_output_df = pd.DataFrame(model_output)
print("Model output columns:", model_output_df.columns.tolist())
print("Labels dataset columns:", labels_dataset.columns.tolist())

# Function to clean special tokens from text
def clean_tokens(text):
    if pd.isna(text):
        return text
    # Remove [CLS], [SEP], [PAD], [MASK] etc.
    text = re.sub(r'\[CLS\]|\[SEP\]|\[PAD\]|\[MASK\]|\[UNK\]', '', text)
    # Remove <token_name> style tokens (e.g., <hedge_up>, < hedge _ up >)
    text = re.sub(r'<\s*[\w_]+(\s+[\w_]+)*\s*>', '', text)
    # Clean up extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Clean source and recover columns
model_output_df['source'] = model_output_df['source'].apply(clean_tokens)
model_output_df['recover'] = model_output_df['recover'].apply(clean_tokens)

# Merge to create universal dataframe with source, recover, id, and label
universal_df = model_output_df.merge(
    labels_dataset[['id', 'class']],  # adjust column name if different
    on='id',
    how='inner'
)

# Select and reorder columns: source, recover, id, label
universal_df = universal_df[['id', 'source', 'recover', 'class']]
print(f"\nUniversal dataset shape: {universal_df.shape}")
universal_df

Model output columns: ['id', 'recover', 'reference', 'source']
Labels dataset columns: ['Unnamed: 0', 'id', 'source', 'recover', 'source_score', 'recover_score', 'class']

Universal dataset shape: (831, 4)


,id,source,recover,class
0,306d294a-0134-4fe3-b985-dba06380fc26,idk about you guys but i got every achievement...,idk about you guys but i think i got got every...,non-suicide
1,ee9577bc-781f-4082-bf48-5079d329e692,dude my teacher just gave us smarties this is ...,dude my teacher just perhaps gave us smarties ...,non-suicide
2,a6d95001-4daf-4744-b326-26f044c8ff06,coincidentally i see one of my closer friends ...,coincidentally i see one of my closer friends ...,non-suicide
3,2ec5dcec-c564-45f3-b2d0-0d45906b4eb5,"didn ' t ask me for permission, just shared it...","didn ' t ask me for permission, just shared it...",non-suicide
4,3314d13a-f7e5-4284-aff9-e1ee5b3ef10b,the bee community is supporting you in this ca...,the bee community is somewhat supporting you t...,non-suicide
...,...,...,...,...
826,6025e6e8-2f45-4a3b-9fab-d4f4c071ea50,i have covid i got covid and i feel sick but i...,i have covid i got covid and i think feel sick...,non-suicide
827,9c28b667-bcc7-4346-a8b6-20023e5dae9c,i would get a job and pay my parents back for ...,i would get a job and pay my parents back for ...,suicide
828,9b5ec268-b38a-4fa2-a872-1fbbace85859,that ' s more pollen than you and i will see i...,that ' s more pollen than you and i might will...,non-suicide
829,27a021b4-c338-4e1a-ac91-418097f59cda,"i cant eat, i cant sleep but am constantly tired.","i cant eat, i might can sleep but am am consta...",suicide


In [20]:
df = pd.read_csv('Suicide_Dataset_Ann.csv')
females = df[(df['gender'] == 'F') & (df['confidence'] >= .99999)]
females.head()

,Unnamed: 0.1,Unnamed: 0,text,class,gender,confidence
97,97,144,I'm going to wait for a version of senorita In...,non-suicide,F,0.999997
165,165,250,I want to make out with Leela in the Planet Ex...,non-suicide,F,0.999997
224,224,330,i have the horni disease and the only cure is ...,non-suicide,F,0.999995
313,313,468,"Who here likes oldies? (Music) Like 50s, 60s, ...",non-suicide,F,0.999999
351,351,522,MotherI’m sorry but I can’t take it anymore. \...,suicide,F,1.000000


In [23]:
import uuid

universal_df['gender'] = 'M'

females_df = females[['text', 'class']].copy()
females_df['id'] = [str(uuid.uuid4()) for _ in range(len(females_df))]
females_df = females_df.rename(columns={'text': 'source'})
females_df['recover'] = females_df['source']
females_df['gender'] = 'F'
females_df = females_df.sample(n=len(universal_df))

universal_df = pd.concat([universal_df, females_df[['id', 'source', 'recover', 'class', 'gender']]], ignore_index=True)
universal_df

,id,source,recover,class,gender
0,306d294a-0134-4fe3-b985-dba06380fc26,idk about you guys but i got every achievement...,idk about you guys but i think i got got every...,non-suicide,M
1,ee9577bc-781f-4082-bf48-5079d329e692,dude my teacher just gave us smarties this is ...,dude my teacher just perhaps gave us smarties ...,non-suicide,M
2,a6d95001-4daf-4744-b326-26f044c8ff06,coincidentally i see one of my closer friends ...,coincidentally i see one of my closer friends ...,non-suicide,M
3,2ec5dcec-c564-45f3-b2d0-0d45906b4eb5,"didn ' t ask me for permission, just shared it...","didn ' t ask me for permission, just shared it...",non-suicide,M
4,3314d13a-f7e5-4284-aff9-e1ee5b3ef10b,the bee community is supporting you in this ca...,the bee community is somewhat supporting you t...,non-suicide,M
...,...,...,...,...,...
1657,cdfc275d-a824-4fc1-8c88-4d9fc9388e65,Someone said Cardi B's nipple looks like dusty...,Someone said Cardi B's nipple looks like dusty...,non-suicide,F
1658,08925926-d55c-4790-8bc4-1831848e474e,Are you a girl gamer? A girl.... AND a gamer? ...,Are you a girl gamer? A girl.... AND a gamer? ...,non-suicide,F
1659,cad7da52-7f6a-4d29-9347-5f92b08fcf00,Gunna make the funny When you do *relatable th...,Gunna make the funny When you do *relatable th...,non-suicide,F
1660,e82025bc-6bdd-4d63-b659-1d17c00935e6,Reasons why 2021 will be my year \- I say I'll...,Reasons why 2021 will be my year \- I say I'll...,non-suicide,F


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

males_df = universal_df[universal_df['gender'] == 'M']
females_df = universal_df[universal_df['gender'] == 'F']

vectorizer = TfidfVectorizer(max_features=5000)

X_males = vectorizer.fit_transform(males_df['source'])
y_males = males_df['class']
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_males, y_males, test_size=0.2, random_state=42)

model_males = LogisticRegression(max_iter=1000)
model_males.fit(X_train_m, y_train_m)
pred_males = model_males.predict(X_test_m)

print("Males Model:")
print(f"Accuracy: {accuracy_score(y_test_m, pred_males):.4f}")
print(classification_report(y_test_m, pred_males))

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
vectorizer_f = TfidfVectorizer(max_features=5000)

X_females = vectorizer_f.fit_transform(females_df['source'])
y_females = females_df['class']
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_females, y_females, test_size=0.2, random_state=42)

model_females = LogisticRegression(max_iter=1000)
model_females.fit(X_train_f, y_train_f)
pred_females = model_females.predict(X_test_f)

print("Females Model:")
print(f"Accuracy: {accuracy_score(y_test_f, pred_females):.4f}")
print(classification_report(y_test_f, pred_females))